# Cassandra

En **Cassandra** vamos a crear 2 tablas, una donde almacenamos los accidentes en formato desnormalizado, y otra que nos servirá para almacenar el número de accidientes por tipo.

En la tabla que almacena los accidentes vamos a utilizar como **PARTITION KEY** el número de expediente y como **CLUSTERING KEY** la fecha del accidente que nos permitirá ordenar la información tal y como nos interesa para nuestro caso de uso.

Adiccionalmente tendremos un índice en la tabla de accidentes que nos permitirá localizar la información por Distrito

<br><br> 

<img src="images/Cassandra.png" width=600 height=500>

<br><br> 

## Carga de datos en Pandas

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_excel("data/accidentes.xlsx", engine='openpyxl')
df.info()

In [ ]:
df.sample(10)

## Creacción de Estructura en Cassandra

In [ ]:
%load_ext cql

In [ ]:
%%cql
DROP KEYSPACE accidentes;

In [ ]:
%%cql
CREATE KEYSPACE accidentes 
WITH replication = {'class':'SimpleStrategy', 'replication_factor': 1};

In [ ]:
%cql USE accidentes;

In [ ]:
%%cql 
CREATE TABLE acum_tipo_accidente (
    tipo_accidente          text,
    total         counter,
    PRIMARY KEY (tipo_accidente)
)

Crea la tabla accidentes de forma que se muestren los datos estén ordenados por fecha

In [ ]:
%%cql 
CREATE TABLE accidentes (
    num_expediente   text,
    fecha            date,
    distrito         text,
    tipo_accidente   text,
    sexo             text,
    PRIMARY KEY <<COMPLETA EL CODIGO>>
)
WITH CLUSTERING ORDER BY (fecha DESC);

Crea un índice para que puedas hacer búsquedas por distrito

In [ ]:
%%cql
CREATE INDEX <<COMPLETA EL CODIGO>>

## Carga de datos en Cassandra

In [ ]:
from cassandra.cluster import Cluster, BatchStatement, ConsistencyLevel
cluster = Cluster()
session = cluster.connect('accidentes')

In [ ]:
import dateutil

def insert_accidentes(df):
    
    sql_insert = """
INSERT INTO accidentes (
num_expediente,
fecha,
distrito,
tipo_accidente,
sexo
) VALUES (%s, %s, %s, %s, %s)
"""

    for index in df.index:
        
        data = [
            df["num_expediente"][index],
            (df['fecha'][index]).date(),
            df["distrito"][index],
            df["tipo_accidente"][index],
            df["sexo"][index]
        ]
        
        session.execute(sql_insert, data)             
        session.execute("UPDATE acum_tipo_accidente SET total = total + 1 WHERE tipo_accidente = %s", 
                        [df["tipo_accidente"][index]]
        )

In [ ]:
insert_accidentes(df)

## Resolución del caso de uso

### Datos del número de expediente 2020S000044

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="images/Cassandra-1.png">

### Accidentes que ocurren en el distrito de Moratalaz ordenados por fecha

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="images/Cassandra-2.png">

### Número de accidentes por tipo

In [ ]:
%%cql 
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="images/Cassandra-3.png">

### Número de accidentes cuyo tipo es colisión frontal

In [ ]:
%%cql
<<COMPLETA EL CODIGO>>

Comprueba que te muestra el siguiente resultado:

<img src="images/Cassandra-4.png">